# Data Loading

In [1]:
#Install Important Libs
%pip install -q datasets transformers pydantic



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import Optional, Literal
from pydantic import BaseModel, Field

# Environment Check before running the code

In [4]:
import importlib
from typing import Optional, Literal
from pydantic import BaseModel

# --- Safe optional imports ---
if importlib.util.find_spec("torch"):
    import torch
else:
    torch = None

if importlib.util.find_spec("datasets"):
    import datasets
else:
    datasets = None

if importlib.util.find_spec("transformers"):
    import transformers
else:
    transformers = None


class LibraryVersions(BaseModel):
    torch: str = " Not Installed"
    datasets: str = " Not Installed"
    transformers: str = " Not Installed"


class GPUInfo(BaseModel):
    device: Literal["GPU", "CPU", "No Device Detected"]
    cuda_version: Optional[str] = None
    gpu_name: Optional[str] = None
    memory_gb: Optional[float] = None
    message: Optional[str] = None


class EnvironmentCheck:
    """Environment checker with Pydantic models."""

    def __init__(self) -> None:
        self.torch_version = torch.__version__ if torch else None
        self.datasets_version = datasets.__version__ if datasets else None
        self.transformers_version = transformers.__version__ if transformers else None

    def get_versions(self) -> LibraryVersions:
        """Return installed library versions as a Pydantic model."""
        return LibraryVersions(
            torch=self.torch_version or " Not Installed",
            datasets=self.datasets_version or " Not Installed",
            transformers=self.transformers_version or " Not Installed",
        )

    def check_gpu(self) -> GPUInfo:
        """Return GPU details as a Pydantic model."""
        if not torch:
            return GPUInfo(device="No Device Detected")

        if torch.cuda.is_available():
            props = torch.cuda.get_device_properties(0)
            return GPUInfo(
                device="GPU",
                cuda_version=torch.version.cuda,
                gpu_name=torch.cuda.get_device_name(0),
                memory_gb=round(props.total_memory / (1024**3), 2),
            )
        return GPUInfo(device="CPU", message="No GPU detected")


# --- Usage for manual run ---
if __name__ == "__main__":
    env = EnvironmentCheck()
    print("Library Versions:", env.get_versions().model_dump())
    print("Device Info:", env.check_gpu().model_dump())


Library Versions: {'torch': '2.8.0.dev20250319+cu128', 'datasets': '4.0.0', 'transformers': '4.55.4'}
Device Info: {'device': 'GPU', 'cuda_version': '12.8', 'gpu_name': 'NVIDIA L4', 'memory_gb': 22.06, 'message': None}


# Data Loading

In [6]:
from typing import Optional
from pydantic import BaseModel, Field
from datasets import load_dataset, DatasetDict


class DatasetConfig(BaseModel):
    """Configuration model for dataset loading."""
    dataset_name: str = Field(..., description="Name of the dataset to load (e.g., 'wikitext').")
    subset_name: str = Field(..., description="Subset/config name (e.g., 'wikitext-2-raw-v1').")


class DatasetLoader:
    """Wrapper class for loading HuggingFace datasets."""

    def __init__(self, config: DatasetConfig) -> None:
        self.config = config

    def load_data(self) -> DatasetDict:
        """Load dataset using HuggingFace."""
        dataset = load_dataset(self.config.dataset_name, self.config.subset_name)
        return dataset


config = DatasetConfig(dataset_name="wikitext", subset_name="wikitext-2-raw-v1")
loader = DatasetLoader(config)
dataset = loader.load_data()

print(dataset)
for i in range(10):
    print(f"Row {i}: {dataset['train'][i]['text']!r}")



DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})
Row 0: ''
Row 1: ' = Valkyria Chronicles III = \n'
Row 2: ''
Row 3: ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Im

# Environment Setup and Dataset Preparation

🔹 Environment Setup

Before training a language model, we must ensure the environment is ready:

Libraries: PyTorch for neural networks, HuggingFace datasets for loading corpora, and transformers for tokenization.

GPU Availability: Training transformers on CPU is not practical. We verified CUDA version, GPU type, and VRAM size.

Reproducibility: Using a structured check (EnvironmentCheck), we can confirm dependencies and hardware setup are consistent across machines.

🔹 Dataset Preparation in LLM Training

Large-scale models are trained on diverse, massive corpora:

🌍 Common Crawl → large-scale web scrape (cleaned and deduplicated).

📚 Books → public-domain and licensed.

📝 Wikipedia → factual, well-structured text.

💻 Code → GitHub, forums, Q&A.

🧪 Research Articles → scientific sources like arXiv and PubMed.

Such datasets often reach billions to trillions of tokens. Preprocessing includes deduplication, filtering, and quality checks.

🔹 Our Choice for EduMoE

For this educational project, we use:

✅ WikiText-2 (raw):

~2M tokens — small enough for fast experiments.

Natural, factual prose similar to real-world corpora.

Already split into train, validation, and test.

Includes quirks like blank lines, which helps mimic real preprocessing challenges.

This dataset is ideal for learning the mechanics of LLM training without overwhelming compute resources.